In [ ]:
import rasterio
from rasterio.merge import merge
from rasterio.io import MemoryFile
import requests
import os
import tempfile
from io import BytesIO

Merging GeoTIFFs...
Writing merged GeoTIFF to: merged_geotiff_zh.tif
Merging completed successfully!
Temporary files cleaned up.


## Download Raster Data
To illustrate the visualization of raster, we download [SWISSIMAGE](https://www.swisstopo.admin.ch/en/orthoimage-swissimage-10) data, which is a composition of color aerial photographs over Switzerland with a ground resolution of 10 cm. The data is provided by the Swiss Federal Office of Topography - swisstopo.

[SWISSIMAGE](https://www.swisstopo.admin.ch/en/orthoimage-swissimage-10) stores the orthophotos split into a mosaic of individual files. Here, we download the images covering Zurich. The individual download links are stored in [orthophoto_urls.txt](orthophoto_urls.txt).

In [ ]:
url_file_path = "orthophoto_urls.txt"
with open(url_file_path, 'r') as file:
    lines = file.readlines()
    filenames = [line.strip() for line in lines if line.strip()]

src_files_to_mosaic = []
temp_files = []
try:
    for url in urls:
        print(f"Downloading GeoTIFF from: {url}")
        geotiff_bytes = download_geotiff(url)
        
        # Create a temporary file
        temp = tempfile.NamedTemporaryFile(delete=False, suffix='.tif')
        temp.write(geotiff_bytes)
        temp.close()
        temp_files.append(temp.name)
        
        # Open the GeoTIFF with Rasterio
        src = rasterio.open(temp.name)
        src_files_to_mosaic.append(src)
    
    print("Merging GeoTIFFs...")
    mosaic, out_trans = merge(src_files_to_mosaic)
    
    # Update metadata
    out_meta = src_files_to_mosaic[0].meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans
    })
    
    print(f"Writing merged GeoTIFF to: {output_path}")
    with rasterio.open(output_path, "w", **out_meta) as dest:
        dest.write(mosaic)
    
    print("Merging completed successfully!")

finally:
    # Cleanup: Close datasets and remove temporary files
    for src in src_files_to_mosaic:
        src.close()
    for temp_path in temp_files:
        os.remove(temp_path)
    print("Temporary files cleaned up.")



def download_geotiff(url):
    """
    Downloads a GeoTIFF from a URL and returns the content as bytes.
    """
    response = requests.get(url)
    response.raise_for_status()  # Raise an error for bad status codes
    return response.content


def merge_geotiffs_temp_files(urls, output_path):
    """
    Merges multiple GeoTIFFs from URLs into a single GeoTIFF using temporary files.
    
    Parameters:
        urls (list): List of GeoTIFF URLs.
        output_path (str): Path to save the merged GeoTIFF.
    """
    src_files_to_mosaic = []
    temp_files = []

    try:
        for url in urls:
            print(f"Downloading GeoTIFF from: {url}")
            geotiff_bytes = download_geotiff(url)
            
            # Create a temporary file
            temp = tempfile.NamedTemporaryFile(delete=False, suffix='.tif')
            temp.write(geotiff_bytes)
            temp.close()
            temp_files.append(temp.name)
            
            # Open the GeoTIFF with Rasterio
            src = rasterio.open(temp.name)
            src_files_to_mosaic.append(src)
        
        print("Merging GeoTIFFs...")
        mosaic, out_trans = merge(src_files_to_mosaic)
        
        # Update metadata
        out_meta = src_files_to_mosaic[0].meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": mosaic.shape[1],
            "width": mosaic.shape[2],
            "transform": out_trans
        })
        
        print(f"Writing merged GeoTIFF to: {output_path}")
        with rasterio.open(output_path, "w", **out_meta) as dest:
            dest.write(mosaic)
        
        print("Merging completed successfully!")
    
    finally:
        # Cleanup: Close datasets and remove temporary files
        for src in src_files_to_mosaic:
            src.close()
        for temp_path in temp_files:
            os.remove(temp_path)
        print("Temporary files cleaned up.")

def read_tif_names(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        # Remove any trailing newline characters and whitespace
        filenames = [line.strip() for line in lines if line.strip()]
    return filenames

if __name__ == "__main__":

    # Example GeoTIFF URLs (Replace these with your actual URLs)
    geotiff_urls = read_tif_names("links_gemeinde_zh.txt")
    
    # Output path for the merged GeoTIFF
    merged_output = "merged_geotiff_zh.tif"
    
    # Approach 2: Temporary Files
    # Note: More memory-efficient, suitable for larger GeoTIFFs
    try:
         merge_geotiffs_temp_files(geotiff_urls, merged_output)
    except Exception as e:
         print(f"An error occurred during temporary file merging: {e}")

In [16]:
with rasterio.open("out/output_cog_small.tif") as src:
    # Read basic metadata
    metadata = src.meta

metadata

{'driver': 'GTiff',
 'dtype': 'uint8',
 'nodata': None,
 'width': 20493,
 'height': 40293,
 'count': 3,
 'crs': CRS.from_wkt('PROJCS["WGS 84 / Pseudo-Mercator",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Mercator_1SP"],PARAMETER["central_meridian",0],PARAMETER["scale_factor",1],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],EXTENSION["PROJ4","+proj=merc +a=6378137 +b=6378137 +lat_ts=0 +lon_0=0 +x_0=0 +y_0=0 +k=1 +units=m +nadgrids=@null +wktext +no_defs"],AUTHORITY["EPSG","3857"]]'),
 'transform': Affine(0.14777180646772103, 0.0, 945949.9074506613,
        0.0, -0.14777180646772103, 6006946.855658049)}

In [17]:
from osgeo import gdal
import os

def reproject_to_cog(
    src_tiff_path,
    dst_cog_path,
    target_crs='EPSG:3857',  # Example: Web Mercator
    tile_size=512,
    compression='DEFLATE',
    num_threads='ALL_CPUS'
):
    """
    Reprojects a GeoTIFF to the target CRS and converts it to a Cloud Optimized GeoTIFF (COG).
    
    Parameters:
        src_tiff_path (str): Path to the source GeoTIFF.
        dst_cog_path (str): Path to save the output COG.
        target_crs (str): Target Coordinate Reference System (e.g., 'EPSG:3857').
        tile_size (int): Tile size for internal tiling.
        compression (str): Compression algorithm (e.g., 'DEFLATE', 'LZW').
        num_threads (str): Number of threads to use (e.g., 'ALL_CPUS').
    """
    
    # Define creation options for COG
    creation_options = [
        'BLOCKSIZE={}'.format(tile_size),
        f'COMPRESS={compression}',
        f'NUM_THREADS={num_threads}',
        'BIGTIFF=IF_SAFER',
        'OVERVIEWS=IGNORE_EXISTING'
    ]
    
    # Open the source dataset
    src_ds = gdal.Open(src_tiff_path)
    if src_ds is None:
        raise FileNotFoundError(f"Unable to open source GeoTIFF: {src_tiff_path}")
    
    # Define the target CRS
    target_crs_proj = target_crs
    
    # Perform the reprojection and COG creation
    print("Reprojecting and converting to COG...")
    dst_ds = gdal.Warp(
        dst_cog_path,
        src_ds,
        dstSRS=target_crs_proj,
        format='COG',
        creationOptions=creation_options,
        multithread=True,
    )
    
    if dst_ds is None:
        raise RuntimeError("gdal.Warp failed to create the COG.")
    
    # Flush data to disk
    dst_ds = None
    src_ds = None
    print(f"COG created successfully at: {dst_cog_path}")

if __name__ == "__main__":
    # Example usage
    src_tiff = merged_output  # Replace with your source GeoTIFF path
    dst_cog = 'out/output_cog_small_jpeg.tif'  # Replace with desired output path
    
    # Ensure the output directory exists
    os.makedirs(os.path.dirname(dst_cog), exist_ok=True)
    
    # Call the function
    try:
        reproject_to_cog(
            src_tiff_path=src_tiff,
            dst_cog_path=dst_cog,
            target_crs='EPSG:3857',  # Example: Web Mercator
            tile_size=512,
            compression='JPEG',
            num_threads='ALL_CPUS'
        )
    except Exception as e:
        print(f"An error occurred: {e}")

Reprojecting and converting to COG...
COG created successfully at: out/output_cog_small_jpeg.tif
